In [1]:
import torch

import numpy as np

from transformers import AutoTokenizer, RobertaForMaskedLM

c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = RobertaForMaskedLM.from_pretrained("FacebookAI/roberta-base")

inputs = tokenizer("The capital of France is <mask>.", return_tensors="pt")
print(inputs)

c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': tensor([[    0,   133,   812,     9,  1470,    16, 50264,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [3]:
with torch.no_grad():
    logits = model(**inputs).logits
    logits = logits.squeeze()
    
output_tokens = logits.argmax(axis=1)[1:-2]
print(tokenizer.decode(output_tokens))

The capital of France is Paris


In [38]:
labels = tokenizer("The capital of France is Paris.", return_tensors='pt')["input_ids"]
print(labels)
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)
print(labels)

tensor([[   0,  133,  812,    9, 1470,   16, 2201,    4,    2]])
tensor([[-100, -100, -100, -100, -100, -100, 2201, -100, -100]])


In [68]:
output = model(**inputs, labels=labels)
round(float(output.loss),2)

0.12